# Entorno de trabajo: Crear un dataset de noticias de prensa a partir del motor de búsqueda Sophia

Este notebook muestra una manera de extraer noticias desde el motor de búsqueda de _Sophia_, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.


_Sophia_ es un proyecto I+i+D desarrollado en la Universidad Austral de Chile desde 2019. El motor de búsqueda se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnica con este notebook, no dudar en contactarme por Discord (@MatthieuVernier) o por correo mvernier@inf.uach.cl.

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [1]:
!python -V

Python 3.8.10


In [2]:
!pip install elasticsearch==7.16.3

In [6]:
import elasticsearch
from datetime import datetime

In [7]:
elasticsearch.__version__

(7, 16, 3)

## 1. Parámetros de la búsqueda

Características del motor de búsqueda:  
- El formato de las fechas es: YYYY-MM-DD
- El motor de búsqueda incluye noticias por distintos paises, pero se recomienda utilizar "chile" por el momento.
- El motor de búsqueda incluye datos entre el año 2000 y el año 2021.

In [8]:
country="chile"
from_="2021-10-01"
to_="2021-10-08"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword=""
simple_keyword=False

## 2. Parámetros de conexión al motor de búsqueda de Sophia

In [18]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [19]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [20]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      #"must": [
      #  {match: { "text":keyword}}

      #],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [21]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 1651 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [17]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    search = keyword
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date,search
0,21912502.0,chile,radiosago,https://www.radiosago.cl/corte-suprema-desesti...,Corte Suprema desestimó solicitud del Municipi...,Luego que la Tercera Sala de la Corte Suprema ...,2021-10-07,
1,21912503.0,chile,radiosago,https://www.radiosago.cl/28-contagios-y-2-fall...,28 contagios y 2 fallecidos por covid-19 se re...,"Así lo dio a conocer la autoridad sanitaria, d...",2021-10-07,
2,21912771.0,chile,radiosago,https://www.radiosago.cl/mas-de-10-mil-trabaja...,Más de 10 mil trabajadores de la región de Los...,Cabe consinar que a nivel nacional son más 21...,2021-10-06,
3,21912783.0,chile,radiosago,https://www.radiosago.cl/adultos-mayores-y-ser...,Adultos mayores y servicios analizan la posibi...,Con la participación de representantes de adul...,2021-10-05,
4,21912788.0,chile,radiosago,https://www.radiosago.cl/monumentos-nacionales...,Monumentos Nacionales inicia trabajos de norma...,El objetivo es adecuar el sistema eléctrico a...,2021-10-06,
...,...,...,...,...,...,...,...,...
1646,47528526.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Casas de apuestas tienen sus favoritos para el...,La autora francesa Annie Ernaux está entre las...,2021-10-07,
1647,47528527.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,"La pelea de Residente y J Balvin no para: ""No ...",En los últimos días se escribió un nuevo capít...,2021-10-07,
1648,47528529.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,Adele anuncia nueva canción a seis años de su ...,La cantante británica Adele desclasificó el ad...,2021-10-07,
1649,47528530.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/1...,"Revelan el primer adelanto de ""House of the Dr...","Sin anuncio previo, HBO lanzó el primer adelan...",2021-10-07,


- Verifiquemos si existen noticias con la misma URL

In [17]:
df['url'].duplicated().any()

True

- Eliminar las noticias duplicadas

In [18]:
df = df.drop_duplicates(subset='url', keep='first')

In [19]:
df

,id_news,country,media_outlet,url,title,text,date,search
0,21912502.0,chile,radiosago,https://www.radiosago.cl/corte-suprema-desesti...,Corte Suprema desestimó solicitud del Municipi...,Luego que la Tercera Sala de la Corte Suprema ...,2021-10-07,
1,21912503.0,chile,radiosago,https://www.radiosago.cl/28-contagios-y-2-fall...,28 contagios y 2 fallecidos por covid-19 se re...,"Así lo dio a conocer la autoridad sanitaria, d...",2021-10-07,
2,21912771.0,chile,radiosago,https://www.radiosago.cl/mas-de-10-mil-trabaja...,Más de 10 mil trabajadores de la región de Los...,Cabe consinar que a nivel nacional son más 21...,2021-10-06,
3,21912783.0,chile,radiosago,https://www.radiosago.cl/adultos-mayores-y-ser...,Adultos mayores y servicios analizan la posibi...,Con la participación de representantes de adul...,2021-10-05,
4,21912788.0,chile,radiosago,https://www.radiosago.cl/monumentos-nacionales...,Monumentos Nacionales inicia trabajos de norma...,El objetivo es adecuar el sistema eléctrico a...,2021-10-06,
...,...,...,...,...,...,...,...,...
1081,47525899.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/10/0...,Piden prisión preventiva para dos exdirectores...,Este miércoles continuó en el Séptimo Juzgado ...,2021-10-07,
1315,47525656.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/10/0...,Elisa Loncon estima que Constitución comenzará...,Tal como lo proyectó el vicepresidente Jaime B...,2021-10-05,
1316,47525657.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/10/0...,Cuarto retiro: Letelier se abre a aprobarlo y ...,Este miércoles la Comisión de Constitución del...,2021-10-05,
1320,47525661.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/10/0...,Se abren investigaciones a líderes involucrado...,Tras la polémica revelación de los llamados Pa...,2021-10-05,


- Guardar los archivos en un CSV

In [21]:
nombre_archivo=keyword.replace(' ', '')+"_"+"los_lagos"+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
_los_lagos_2021-10-01_2021-10-08.csv


- Realizar algunas consultas SQL sobre el dataset

In [22]:
from pandasql import sqldf 

In [23]:
result = sqldf("SELECT media_outlet,count(*) FROM df GROUP BY media_outlet ORDER BY count(*) DESC")
result

,media_outlet,count(*)
0,laestrelladechiloe,102
1,radiosago,89
2,elllanquihue,84
3,elaustral,84
4,elheraldoaustral,43
5,elinsular,36
6,elrepuertero,35
7,seminariolocal,24
8,elvacanudo,23
9,radioacogida,17
